In [2]:
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from denoising_score import ScoreNet, dsm

#訓練データ
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download = True)
#検証データ
test_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=False,
                                           transform=transforms.ToTensor(),
                                           download = True)

Failed to download (trying next):
<urlopen error [Errno 60] Operation timed out>



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 60] Operation timed out>



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 60] Operation timed out>



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 60] Operation timed out>



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [7]:
batch_size = 600
train_loader = torch.utils.data.DataLoader(
    train_dataset,         
    batch_size=batch_size,  
    shuffle=True,          
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
)

In [41]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
epochs = 1000
sigma = 0.1
score = ScoreNet(784)
optimizer = torch.optim.Adam(score.parameters(), lr=0.005)
for epoch in range(epochs):
    for x, _ in train_loader:
        x = x.view(600, -1, 28*28).to(device)
        x_tilde = x + torch.randn_like(x) * sigma
        target = -(x_tilde - x)/(sigma**2)
        s_theta = score(x)
        loss = dsm(s_theta, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch == 0 or (epoch+1) %100 == 0:
        print(f"epoch={epoch+1}, loss={loss.item()}")

cpu


KeyboardInterrupt: 